In [1]:
import sys
from pathlib import Path
wdir = Path.cwd().parent.absolute()
sys.path.insert(1, str(wdir))

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_ChanRed, CapsEEGNet, TCNet, EEGNet_WT, TCNet_EMD
import torch
import os

In [2]:
from preprocess.preprocess_DREAMER import DREAMERDataset

# dataset = DREAMERDataset('', "valence", tfr=2)

In [3]:
model = EEGNet(nb_classes = 2, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = 0
for name, param in model.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: block1.0.weight, Parameters: 1024
Layer: block1.1.weight, Parameters: 64
Layer: block1.1.bias, Parameters: 64
Layer: block1.2.weight, Parameters: 7168
Layer: block1.3.weight, Parameters: 512
Layer: block1.3.bias, Parameters: 512
Layer: block2.0.depthwise.weight, Parameters: 8192
Layer: block2.0.pointwise.weight, Parameters: 32768
Layer: block2.1.weight, Parameters: 64
Layer: block2.1.bias, Parameters: 64
Layer: dense.weight, Parameters: 512
Layer: dense.bias, Parameters: 2
Total Parameters: 50946


In [4]:
model = EEGNet_WT(nb_classes = 2, Chans = 14, InnerChans=18, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout', nb_freqs=2)

total_params = 0
for name, param in model.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: chan_reduction.weight, Parameters: 504
Layer: chan_reduction.bias, Parameters: 36
Layer: block1.0.weight, Parameters: 2048
Layer: block1.1.weight, Parameters: 64
Layer: block1.1.bias, Parameters: 64
Layer: block1.2.weight, Parameters: 9216
Layer: block1.3.weight, Parameters: 512
Layer: block1.3.bias, Parameters: 512
Layer: block2.0.depthwise.weight, Parameters: 8192
Layer: block2.0.pointwise.weight, Parameters: 32768
Layer: block2.1.weight, Parameters: 64
Layer: block2.1.bias, Parameters: 64
Layer: dense.weight, Parameters: 512
Layer: dense.bias, Parameters: 2
Total Parameters: 54558


In [5]:
x = torch.randn(8, 14, 2, 128)
out = model(x)
print(out.shape)

torch.Size([8, 2])


/Users/juliengadonneix/anaconda3/envs/intern3A/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1712608633180/work/aten/src/ATen/native/Convolution.cpp:1032.)
  return F.conv2d(input, weight, bias, self.stride,


In [6]:
model_SEED = EEGNet_ChanRed(nb_classes = 3, Chans = 62, InnerChans=24, Samples = 200, dropoutRate = 0.1, kernLength = 20, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = 0
for name, param in model_SEED.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: chan_reduction.weight, Parameters: 1488
Layer: chan_reduction.bias, Parameters: 24
Layer: block1.0.weight, Parameters: 1280
Layer: block1.1.weight, Parameters: 64
Layer: block1.1.bias, Parameters: 64
Layer: block1.2.weight, Parameters: 12288
Layer: block1.3.weight, Parameters: 512
Layer: block1.3.bias, Parameters: 512
Layer: block2.0.depthwise.weight, Parameters: 8192
Layer: block2.0.pointwise.weight, Parameters: 32768
Layer: block2.1.weight, Parameters: 64
Layer: block2.1.bias, Parameters: 64
Layer: dense.weight, Parameters: 1152
Layer: dense.bias, Parameters: 3
Total Parameters: 58475


In [7]:
x = torch.randn(8, 1, 62, 200)
y = model_SEED(x)
print(y.size())

torch.Size([8, 3])


In [8]:
model_capseegnet = CapsEEGNet(nb_classes=2, Chans=14)

total_params = 0
for name, param in model_capseegnet.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: block_1_2.0.weight, Parameters: 512
Layer: block_1_2.1.weight, Parameters: 8
Layer: block_1_2.1.bias, Parameters: 8
Layer: block_1_2.3.weight, Parameters: 224
Layer: block_1_2.4.weight, Parameters: 16
Layer: block_1_2.4.bias, Parameters: 16
Layer: primaryCaps.caps.weight, Parameters: 24576
Layer: primaryCaps.caps.bias, Parameters: 256
Layer: primaryCaps.caps2.weight, Parameters: 69632
Layer: primaryCaps.caps2.bias, Parameters: 256
Layer: emotionCaps.W, Parameters: 1048576
Layer: fc.weight, Parameters: 16
Layer: fc.bias, Parameters: 1
Total Parameters: 1144097


In [9]:
x = torch.randn(8, 1, 14, 128)
y = model_capseegnet(x)
print(y.size())
print(y)

torch.Size([8, 2])
tensor([[0.4997, 0.5003],
        [0.5012, 0.4988],
        [0.5007, 0.4993],
        [0.4995, 0.5005],
        [0.4992, 0.5008],
        [0.4998, 0.5002],
        [0.4992, 0.5008],
        [0.4997, 0.5003]], grad_fn=<SoftmaxBackward0>)


In [10]:
model_tcnet = TCNet_EMD(nb_classes=2, Chans=14, nb_freqs=2, kern_emd=12)


total_params = 0
for name, param in model_tcnet.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: emd_expansion.weight, Parameters: 1728
Layer: emd_expansion.bias, Parameters: 48
Layer: PatchPartition.weight, Parameters: 5376
Layer: PatchPartition.bias, Parameters: 32
Layer: stages.0.WMSA.pos_embeddings, Parameters: 9
Layer: stages.0.WMSA.linear1.weight, Parameters: 3072
Layer: stages.0.WMSA.linear1.bias, Parameters: 96
Layer: stages.0.WMSA.linear2.weight, Parameters: 1024
Layer: stages.0.WMSA.linear2.bias, Parameters: 32
Layer: stages.0.SWMSA.pos_embeddings, Parameters: 9
Layer: stages.0.SWMSA.linear1.weight, Parameters: 3072
Layer: stages.0.SWMSA.linear1.bias, Parameters: 96
Layer: stages.0.SWMSA.linear2.weight, Parameters: 1024
Layer: stages.0.SWMSA.linear2.bias, Parameters: 32
Layer: stages.0.ln.weight, Parameters: 32
Layer: stages.0.ln.bias, Parameters: 32
Layer: stages.0.MLP.ff.0.weight, Parameters: 4096
Layer: stages.0.MLP.ff.0.bias, Parameters: 128
Layer: stages.0.MLP.ff.2.weight, Parameters: 4096
Layer: stages.0.MLP.ff.2.bias, Parameters: 32
Layer: stages.1.WMSA.pos

In [11]:
x = torch.randn(8, 14, 3, 128)
out = model_tcnet(x)
print(out.shape)

torch.Size([8, 2])


In [12]:
print(os.cpu_count())
print(torch.cuda.device_count()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)
print(device.type == 'cuda')
if device.type == 'cuda':
    properties = torch.cuda.get_device_properties(device)
    print(properties)

8
0
mps
False


In [13]:
import subprocess
import plistlib

def get_gpu_info():
    """Get information about the GPU on macOS."""
    # Run system_profiler command and capture the output
    result = subprocess.run(['system_profiler', '-xml', 'SPDisplaysDataType'], capture_output=True)
    if result.returncode == 0:
        # Parse the XML output
        display_info = plistlib.loads(result.stdout)
        # Extract GPU information
        gpu_count = len(display_info[0]['_items'])
        return gpu_count
    else:
        print("Error:", result.stderr)
        return None

gpu_count = get_gpu_info()
if gpu_count is not None:
    print("Number of GPUs:", gpu_count)


Number of GPUs: 1


In [14]:
import time
import torch

start_time = time.time()

# syncrocnize time with cpu, otherwise only time for oflaoding data to gpu would be measured
torch.mps.synchronize()

a = torch.ones(4000,4000, device="mps")
for _ in range(200):
   a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.2951338291168213


In [15]:
t = torch.randn(8,8)
print(t)
torch.reshape(t, (-1, 2, 8))

tensor([[-2.3310, -0.7779, -0.8517,  0.3107,  0.4501, -1.4042,  0.2066,  0.4133],
        [-1.2149,  0.9095,  0.6724,  1.6511,  0.6922, -1.2538, -0.2289,  1.0141],
        [-0.2060, -1.7363, -2.1075, -1.0517,  1.5688,  0.3479, -0.9671, -1.6473],
        [-0.4754,  0.3361,  1.8254, -0.0305,  1.5051,  0.6922,  0.7587, -1.1033],
        [ 0.3410,  0.0603,  0.2560,  0.4281,  1.0576,  0.0429, -1.6011, -0.1525],
        [-1.6114, -1.3564,  1.1681,  0.1758, -0.0886,  1.2342, -0.1580, -0.5711],
        [ 0.7895,  0.1875, -0.6011, -0.7455,  1.7527,  0.6073, -1.6166, -0.7140],
        [-0.6812, -0.4708, -0.3051, -1.1373, -1.4031, -1.4501, -0.9560,  2.2691]])


tensor([[[-2.3310, -0.7779, -0.8517,  0.3107,  0.4501, -1.4042,  0.2066,
           0.4133],
         [-1.2149,  0.9095,  0.6724,  1.6511,  0.6922, -1.2538, -0.2289,
           1.0141]],

        [[-0.2060, -1.7363, -2.1075, -1.0517,  1.5688,  0.3479, -0.9671,
          -1.6473],
         [-0.4754,  0.3361,  1.8254, -0.0305,  1.5051,  0.6922,  0.7587,
          -1.1033]],

        [[ 0.3410,  0.0603,  0.2560,  0.4281,  1.0576,  0.0429, -1.6011,
          -0.1525],
         [-1.6114, -1.3564,  1.1681,  0.1758, -0.0886,  1.2342, -0.1580,
          -0.5711]],

        [[ 0.7895,  0.1875, -0.6011, -0.7455,  1.7527,  0.6073, -1.6166,
          -0.7140],
         [-0.6812, -0.4708, -0.3051, -1.1373, -1.4031, -1.4501, -0.9560,
           2.2691]]])